<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
''' Update notes by PRK Nov 10:
        Taken out zipcode column entirely and added three more column removals (have commented in front)
        Also commented out the print order'''

' Update notes by PRK Nov 10:\n        Taken out zipcode column entirely and added three more column removals (have commented in front)\n        Also commented out the print order'

In [93]:
# Importing the required libraries and methods
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
import datetime as dt
# Importing the dataset
filename = '../Data/listings.csv'
reviews_filename = '../Data/reviews_cleaned.csv'
data = pd.read_csv(filename)
reviews = pd.read_csv(reviews_filename, names = ['listing_id', 'comments'])
# print(data.info)
# print(list(data))
# print(list(data)[43])
# print(list(data)[87])
# print(list(data)[88])

/var/folders/dh/pf9jys0d4_s_bxm_6zl1fbdh0000gn/T/ipykernel_1193/142444434.py:10: DtypeWarning: Columns (43,87,88) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(filename)


In [94]:
# Taking out the unwanted columns
print(len(data.columns))
#exit()
data = pd.DataFrame.drop(data, columns=[
    'host_name',
    'notes', # Added PRK
    'host_about', # Added PRK
    'calendar_updated', # Added PRK
    'host_acceptance_rate',
    'description',
    'thumbnail_url',
    'experiences_offered',
    'listing_url',
    'name',
    'summary',
    'space',
    'scrape_id',
    'last_scraped',
    'neighborhood_overview',
    'transit',
    'access',
    'interaction',
    'house_rules',
    'medium_url',
    'picture_url',
    'xl_picture_url',
    'host_url',
    'host_thumbnail_url',
    'host_picture_url',
    'host_acceptance_rate',
    'smart_location',
    'license',
    'jurisdiction_names',
    'street',
    'neighbourhood',
    'country',
    'country_code',
    'host_location',
    'host_neighbourhood',
    'market',
    'is_location_exact',
    'square_feet',
    'weekly_price',
    'monthly_price',
    'availability_30',
    'availability_60',
    'availability_90',
    'availability_365',
    'calendar_last_scraped',
    'first_review',
    'last_review',
    'requires_license',
    'calculated_host_listings_count',
    'host_listings_count',

     #discuss last two
    #'zipcode' # Added PRK
])

96


In [95]:
df_demographics1 = pd.read_excel('../Data/nyc_zcta_census_data.xlsx', sheet_name='b_zctas_2018acs1')
df_demographics1 = df_demographics1[['GEOID2', 'EDU01_E','EDU10_E','INC12_E']]

df_demographics2 = pd.read_excel('../Data/nyc_zcta_census_data.xlsx', sheet_name='b_zctas_2018acs2')
df_demographics2 = df_demographics2[['GEOID2', 'SXAG17_E']]

df_demo = pd.merge(df_demographics1, df_demographics2, on='GEOID2')
df_demo = df_demo.rename(columns = {'GEOID2':'zipcode', 'INC12_E':'INC_median', 'SXAG17_E':'Age_median'})


In [96]:
df_demo['zipcode'] = df_demo['zipcode'].astype(str)

In [98]:
def clean_demo_income(entry):
    if math.isnan(entry):
        return -55
    else:
        return entry
    
def clean_demo_edu(entry):
    if entry == 0:
        return -55
    else:
        return entry
df_demo['INC_median'] = df_demo['INC_median'].apply(clean_demo_income)
df_demo['EDU01_E'] = df_demo['EDU01_E'].apply(clean_demo_edu)
df_demo = df_demo[df_demo['INC_median'] != -55]
df_demo = df_demo[df_demo['EDU01_E'] != -55]

In [99]:
df_demo['EDU10_E'] = df_demo['EDU10_E']/df_demo['EDU01_E'] #ratio

In [100]:
df_demo = df_demo.drop(columns=['EDU01_E'])
df_demo

,zipcode,EDU10_E,INC_median,Age_median
0,10001,0.680178,88526.0,36.7
1,10002,0.339588,35859.0,43.6
2,10003,0.801836,112131.0,32.3
3,10004,0.904201,157645.0,37.1
4,10005,0.884452,173333.0,30.1
...,...,...,...,...
210,11691,0.258174,46147.0,31.1
211,11692,0.234161,44024.0,34.2
212,11693,0.263671,59431.0,39.5
213,11694,0.434377,80976.0,48.5


In [101]:
data = pd.merge(data, df_demo, on='zipcode')

In [102]:
data

,id,host_id,host_since,host_response_time,host_response_rate,host_is_superhost,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,...,review_scores_value,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,reviews_per_month,EDU10_E,INC_median,Age_median
0,2515,2758,2008-09-06,within a few hours,93%,f,3.0,"['email', 'phone', 'facebook', 'reviews', 'jum...",t,t,...,9.0,f,f,strict_14_with_grace_period,f,f,1.41,0.444305,58884.0,35.1
1,8225,2758,2008-09-06,within a few hours,93%,f,3.0,"['email', 'phone', 'facebook', 'reviews', 'jum...",t,t,...,9.0,f,f,strict_14_with_grace_period,f,f,1.32,0.444305,58884.0,35.1
2,49715,226986,2010-09-07,within an hour,100%,t,1.0,"['email', 'phone', 'reviews', 'kba']",t,t,...,10.0,f,f,strict_14_with_grace_period,t,t,1.22,0.444305,58884.0,35.1
3,54466,253385,2010-10-05,NaN,NaN,f,1.0,"['email', 'phone', 'reviews', 'jumio', 'govern...",t,t,...,NaN,f,f,moderate,f,f,NaN,0.444305,58884.0,35.1
4,67397,332189,2011-01-01,within an hour,100%,f,1.0,"['email', 'phone', 'facebook', 'reviews', 'kba']",t,t,...,9.0,f,f,moderate,t,t,0.82,0.444305,58884.0,35.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31774,23000751,170603631,2018-01-29,within an hour,100%,t,1.0,"['email', 'phone']",t,f,...,10.0,t,f,strict_14_with_grace_period,f,f,2.39,0.434017,89185.0,49.4
31775,25134800,48983827,2015-11-13,within an hour,100%,f,1.0,"['email', 'phone', 'facebook', 'reviews']",t,f,...,10.0,t,f,flexible,f,f,0.70,0.434017,89185.0,49.4
31776,21405589,155091790,2017-10-18,a few days or more,40%,f,2.0,"['email', 'phone']",t,f,...,NaN,f,f,flexible,f,f,NaN,0.335854,103175.0,40.5
31777,26258351,6402171,2013-05-15,within an hour,90%,f,1.0,"['email', 'phone', 'reviews', 'jumio', 'offlin...",t,f,...,10.0,t,f,moderate,f,f,1.30,0.335854,103175.0,40.5


In [103]:
print('Splitting host verifications')
host_verification_set = set()

Splitting host verifications


In [104]:
def collect_host_verifications(entry):
    entry_list = entry.replace("[", "").replace("]", "").replace("'", "").replace('"', "").replace(" ", "").split(',')
    for verification in entry_list:
        if (verification != "" and verification != 'None'):
            host_verification_set.add(verification +"_verification")

In [105]:
data['host_verifications'].apply(collect_host_verifications)

0        None
1        None
2        None
3        None
4        None
         ... 
31774    None
31775    None
31776    None
31777    None
31778    None
Name: host_verifications, Length: 31779, dtype: object

In [106]:
def generic_verification(entry, v):
    entry_list = str(entry).replace("[", "").replace("]", "").replace("'", "").replace('"', "").replace(" ", "").split(',')
    for verification in entry_list:
        if (verification + "_verification" == v):
            return 1
    return 0

In [107]:
for v in host_verification_set:
    data.insert(len(list(data)), v, 0)
    data[v] = data['host_verifications'].apply(lambda x: generic_verification(x, v))

In [108]:
data = pd.DataFrame.drop(data, columns=['host_verifications'])

In [109]:
def clean_response_rate(entry):
    if (type(entry) == str):
        return entry.replace('%', '')
    else:
        return 0

In [110]:
data['host_response_rate'] = data['host_response_rate'].apply(clean_response_rate)

In [111]:
def clean_superhost(entry):
    if (entry == 't'):
        return 1
    else:
        return 0
data['host_is_superhost'] = data['host_is_superhost'].apply(clean_superhost)
data['host_has_profile_pic'] = data['host_has_profile_pic'].apply(clean_superhost)
data['host_identity_verified'] = data['host_identity_verified'].apply(clean_superhost)
data['has_availability'] = data['has_availability'].apply(clean_superhost)
data['instant_bookable'] = data['instant_bookable'].apply(clean_superhost)
data['is_business_travel_ready'] = data['is_business_travel_ready'].apply(clean_superhost)
data['require_guest_profile_picture'] = data['require_guest_profile_picture'].apply(clean_superhost)
data['require_guest_phone_verification'] = data['require_guest_phone_verification'].apply(clean_superhost)

In [112]:
"""
print(list(data))

print(data['host_verifications'][0])
for v in host_verification_set:
    print(v, " ", data[v][0])
"""
def clean_price(entry):
    if (type(entry) != str and math.isnan(entry)):
        return -55
    entry1 = entry.replace('$', '').replace(',', '')
    if (float(entry1) == 0):
        return -55
    return np.log(float(entry1))

In [113]:
def clean_number(entry):
    if (math.isnan(entry)):
        return 0
    else:
        return entry
def clean_number_removal(entry):
    if (math.isnan(entry)):
        return -55
    else:
        return entry
data['bathrooms'] = data['bathrooms'].apply(clean_number_removal)
data['bedrooms'] = data['bedrooms'].apply(clean_number_removal)
data['beds'] = data['beds'].apply(clean_number_removal)
data = data[data['bathrooms'] != -55]
data = data[data['bedrooms'] != -55]
data = data[data['beds'] != -55]

In [114]:
def reviews_per_month_cleanup(entry):
    if (math.isnan(entry)):
        return 0
    return entry

In [115]:
data['reviews_per_month'] = data['reviews_per_month'].apply(reviews_per_month_cleanup)
data['price'] = data['price'].apply(clean_price)
data = data[data['price'] != -55]
data['extra_people'] = data['extra_people'].apply(clean_price)
data['security_deposit'] = data['security_deposit'].apply(clean_price)
data['cleaning_fee'] = data['cleaning_fee'].apply(clean_price)
def clean_listings_count(entry):
    if (math.isnan(entry)):
        return 1
    return entry
data['host_total_listings_count'] = data['host_total_listings_count'].apply(clean_listings_count)
print("Cleaning the state")
def cleaned_state(entry):
    if (isinstance(entry, str)):
        if (entry.upper() == 'NY' or entry.upper == 'New York'):
            return 'NY'
        else:
            return entry
    elif math.isnan(entry):
        return ''
    else:
        return entry
data['state'] = data['state'].apply(cleaned_state)
data = data[data['state'] == 'NY']
state = {}
def create_state_set(entry):
    if (entry not in state):
        state[entry] = 1
    else:
        state[entry] += 1

Cleaning the state


In [116]:
data['state'].apply(create_state_set)
# print(state)

0        None
1        None
2        None
3        None
4        None
         ... 
31774    None
31775    None
31776    None
31777    None
31778    None
Name: state, Length: 31589, dtype: object

In [117]:
print('Spliting amenities')
amenities_set = set()

Spliting amenities


In [118]:
def collect_amenities(entry):
    entry_list = entry.replace("{", "").replace("}", "").replace("'", "").replace('"', "").replace(" ", "_").split(',')
    for am in entry_list:
        if ('translation_missing' not in am and am != ''):
            amenities_set.add(am)

In [119]:
data['amenities'].apply(collect_amenities)
#print(amenities_set)

0        None
1        None
2        None
3        None
4        None
         ... 
31774    None
31775    None
31776    None
31777    None
31778    None
Name: amenities, Length: 31589, dtype: object

In [120]:
def generic_amenities(entry, amenity):
    entry_list = entry.replace("{", "").replace("}", "").replace("'", "").replace('"', "").replace(" ", "_").split(',')
    for am in entry_list:
        if (am == amenity):
            return 1
    return 0

In [121]:
for amenity in amenities_set:
    data.insert(len(list(data)), amenity, 0)
    data[amenity] = data['amenities'].apply(lambda x: generic_amenities(x, amenity))

/var/folders/dh/pf9jys0d4_s_bxm_6zl1fbdh0000gn/T/ipykernel_1193/182050612.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data.insert(len(list(data)), amenity, 0)
/var/folders/dh/pf9jys0d4_s_bxm_6zl1fbdh0000gn/T/ipykernel_1193/182050612.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data.insert(len(list(data)), amenity, 0)
/var/folders/dh/pf9jys0d4_s_bxm_6zl1fbdh0000gn/T/ipykernel_1193/182050612.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, wh

/var/folders/dh/pf9jys0d4_s_bxm_6zl1fbdh0000gn/T/ipykernel_1193/182050612.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data.insert(len(list(data)), amenity, 0)
/var/folders/dh/pf9jys0d4_s_bxm_6zl1fbdh0000gn/T/ipykernel_1193/182050612.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data.insert(len(list(data)), amenity, 0)
/var/folders/dh/pf9jys0d4_s_bxm_6zl1fbdh0000gn/T/ipykernel_1193/182050612.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, wh

In [122]:
print(data['amenities'][0])
for v in  amenities_set:
    print(v, " ", data[v][0])

{TV,"Cable TV",Internet,Wifi,"Air conditioning",Kitchen,"Free street parking","Buzzer/wireless intercom",Heating,"Family/kid friendly","Smoke detector","Carbon monoxide detector","Fire extinguisher",Essentials,Shampoo,"Lock on bedroom door",Hangers,"Hair dryer",Iron,"Laptop friendly workspace","Children’s books and toys","Window guards","Pack ’n Play/travel crib","Hot water",Microwave,"Coffee maker",Refrigerator,"Dishes and silverware","Cooking basics",Oven,Stove,"Host greets you"}
Beach_essentials   0
Pets_allowed   0
Baby_bath   0
Firm_mattress   0
Essentials   1
Kitchen   1
Host_greets_you   1
Microwave   1
Pool   0
Breakfast   0
Suitable_for_events   0
Lockbox   0
Game_console   0
Pets_live_on_this_property   0
Garden_or_backyard   0
Room-darkening_shades   0
Single_level_home   0
Hair_dryer   1
Indoor_fireplace   0
Accessible-height_bed   0
Wide_clearance_to_bed   0
Fireplace_guards   0
Washer_/_Dryer   0
Family/kid_friendly   1
Dog(s)   0
Refrigerator   1
Cable_TV   1
Gym   0
Ele

In [123]:
#maybe drop the original column??
data = pd.DataFrame.drop(data, columns=['amenities', 'state'])

In [124]:
for col_name in ['property_type', 'bed_type',
                 'room_type', 'neighbourhood_group_cleansed', 'city',
                 'cancellation_policy', 'host_response_time', 'neighbourhood_cleansed']:
    parsed_cols = pd.get_dummies(data[col_name])
    data = data.drop(columns=[col_name])
    data = pd.concat([data, parsed_cols], axis = 1)

In [125]:
# Changing the host_since to number of days until 10 Nov 2018
def clean_host_since(entry):
    if (type(entry) != str and math.isnan(entry)):
        return -55
    return entry
data['host_since'] = data['host_since'].apply(clean_host_since)
data = data[data['host_since'] != -55]
dummy_date = dt.datetime(2018,11,10)
data.host_since = (dummy_date - pd.to_datetime(data.host_since))
data.host_since = data.host_since.apply(lambda x: float(x.days))

In [126]:
for col_name in ['review_scores_rating', 'review_scores_accuracy',
                 'review_scores_cleanliness', 'review_scores_checkin',
                 'review_scores_communication', 'review_scores_location',
                 'review_scores_value']:
    data[col_name] = data[col_name].apply(lambda x: 0 if np.isnan(x) else x)

In [127]:
data[col_name] = data[col_name].apply(lambda x: 0 if np.isnan(x) else x)

In [128]:
 data.isna().any().value_counts()

False    682
dtype: int64

In [129]:
data = pd.DataFrame.drop(data, columns=['zipcode'])
data

,id,host_id,host_since,host_response_rate,host_is_superhost,host_total_listings_count,host_has_profile_pic,host_identity_verified,latitude,longitude,...,Westerleigh,Whitestone,Williamsbridge,Williamsburg,Willowbrook,Windsor Terrace,Woodhaven,Woodlawn,Woodrow,Woodside
0,2515,2758,3717.0,93,0,3.0,1,1,40.799205,-73.953676,...,0,0,0,0,0,0,0,0,0,0
1,8225,2758,3717.0,93,0,3.0,1,1,40.799034,-73.954063,...,0,0,0,0,0,0,0,0,0,0
2,49715,226986,2986.0,100,1,1.0,1,1,40.805821,-73.953403,...,0,0,0,0,0,0,0,0,0,0
3,54466,253385,2958.0,0,0,1.0,1,1,40.802344,-73.956028,...,0,0,0,0,0,0,0,0,0,0
4,67397,332189,2870.0,100,0,1.0,1,1,40.803934,-73.958380,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31774,23000751,170603631,285.0,100,1,1.0,1,0,40.748074,-73.736740,...,0,0,0,0,0,0,0,0,0,0
31775,25134800,48983827,1093.0,100,0,1.0,1,0,40.750137,-73.738566,...,0,0,0,0,0,0,0,0,0,0
31776,21405589,155091790,388.0,40,0,2.0,1,0,40.529143,-74.194719,...,0,0,0,0,0,0,0,0,0,0
31777,26258351,6402171,2005.0,90,0,1.0,1,0,40.547904,-74.210166,...,0,0,0,0,0,0,0,0,0,0


In [130]:
data = data.set_index('id').join(reviews.set_index('listing_id'))
def clean_comments(entry):
    if (type(entry) != str and math.isnan(entry)):
        return 0
    return entry
data['comments'] = data['comments'].apply(clean_comments)
data.to_csv('../Data/data_cleaned.csv')